## DOWNLOADING DATA FROM S3 And ORGANIZING BY Session ID Folder

In [36]:
import boto3
import os

# Initialize the S3 client
s3 = boto3.client('s3')

# Define the bucket name and folder
bucket_name = 'bbti-ai-grading-uat'  # Replace with your bucket name
folder_name = 'input-image/'  # Folder to download
local_download_path = './downloaded_input_image'  # Local folder to save files

# Ensure local download directory exists
os.makedirs(local_download_path, exist_ok=True)

# Initialize paginator to handle pagination
paginator = s3.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=folder_name)

# Iterate over all pages
print(f"Downloading files from folder: {folder_name}")
for page in page_iterator:
    if 'Contents' in page:
        for obj in page['Contents']:
            file_key = obj['Key']
            
            # Skip the folder itself (empty key after prefix)
            if file_key == folder_name:
                continue
            
            # Define local file path
            local_file_path = os.path.join(local_download_path, os.path.relpath(file_key, folder_name))
            
            # Ensure local subdirectories exist
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
            
            # Check if the file already exists locally
            if os.path.exists(local_file_path):
                print(f"File already exists, skipping: {local_file_path}")
                continue
            
            # Download the file if it doesn't exist locally
            print(f"Downloading {file_key} to {local_file_path}")
            s3.download_file(bucket_name, file_key, local_file_path)
print("Download complete!")

File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_BACK_1736298605181.png
File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_BOTTOM_1736298642816.png
File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_FRONT_1736298598144.png
File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_LEFT_1736298613037.png
File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_RIGHT_1736298623295.png
File already exists, skipping: ./downloaded_input_image/04688db2-3a16-40d6-929b-671d70f1202b_INPUT_TOP_1736298638224.png
File already exists, skipping: ./downloaded_input_image/04fad5ad-ac47-4944-910d-07276eec37c5_INPUT_BACK_1736477336681.png
File already exists, skipping: ./downloaded_input_image/04fad5ad-ac47-4944-910d-07276eec37c5_INPUT_BOTTOM_1736477364377.png
File already exists

In [37]:
import os
import shutil

# Define the source folder containing the downloaded images
source_folder = './downloaded_input_image'  # Replace with your folder path
destination_base_folder = './organized_images'  # Folder to organize images into subfolders

# Ensure the destination base folder exists
os.makedirs(destination_base_folder, exist_ok=True)

# Iterate over all files in the source folder
for file_name in os.listdir(source_folder):
    if not file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')):  # Skip non-image files
        continue

    # Extract the session_id from the filename (portion before '_INPUT')
    session_id = file_name.split('_INPUT')[0]
    
    # Create a subfolder for the session_id
    session_folder = os.path.join(destination_base_folder, session_id)
    os.makedirs(session_folder, exist_ok=True)
    
    # Move the file to the respective session_id subfolder
    source_file_path = os.path.join(source_folder, file_name)
    destination_file_path = os.path.join(session_folder, file_name)
    shutil.move(source_file_path, destination_file_path)
    print(f"Moved {file_name} to {session_folder}")

print("Images have been organized into subfolders by session_id!")

Moved 834db200-30cb-4737-bfdc-a14d95e3e863_INPUT_TOP_1736496476515.png to ./organized_images/834db200-30cb-4737-bfdc-a14d95e3e863
Moved daa28793-2bf7-40d8-8fa1-32833875b54c_INPUT_FRONT_1736133048222.png to ./organized_images/daa28793-2bf7-40d8-8fa1-32833875b54c
Moved 1411b366-e95e-491b-b48b-d3995cd74efd_INPUT_BACK.jpg to ./organized_images/1411b366-e95e-491b-b48b-d3995cd74efd
Moved b8b15cad-fe71-4b67-b108-29566e292593_INPUT_BACK_1736218314107.png to ./organized_images/b8b15cad-fe71-4b67-b108-29566e292593
Moved 7a5d7996-ff87-4f60-8741-3a2b0a2b5c06_INPUT_BOTTOM_1736478364687.png to ./organized_images/7a5d7996-ff87-4f60-8741-3a2b0a2b5c06
Moved 314e59c6-b0ae-4f53-afa7-2afd13914c99_INPUT_FRONT_1736133354750.png to ./organized_images/314e59c6-b0ae-4f53-afa7-2afd13914c99
Moved 692bfcb4-4057-4b2c-8913-36b3229fa717_INPUT_BACK_1736131236333.png to ./organized_images/692bfcb4-4057-4b2c-8913-36b3229fa717
Moved e6614a3a-132b-47dd-a772-c358717525c0_INPUT_BACK_1736390630293.png to ./organized_images/

### download data for updated session ids

In [ ]:


# Main execution
if __name__ == "__main__":
    # File paths and configurations
    
    # csv_file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_14thjan_dbdata.csv'  # Path to  CSV
    csv_file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_14thjan_dbdata_datefix.csv'
    date_to_filter = '2025-01-14'  # Date to filter session IDs
    bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
    s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
    local_root_folder = './new_organized_images'  # Local folder for new data

    # Step 1: Get session IDs for January 14, 2025
    new_session_ids = get_new_session_ids(csv_file_path, date_to_filter)
    print(f"Filtered session IDs for {date_to_filter}: {new_session_ids}")

    # Step 2: Download and organize data for the filtered session IDs
    download_and_organize_new_data(new_session_ids, bucket_name, s3_folder_prefix, local_root_folder)

Filtered session IDs for 2025-01-14: ['a832c51a-9669-4eeb-aa43-b4fff74b0698'
 '7680de40-46fb-41c5-95d0-f1062254bd48'
 '4fb94a16-fb78-4b9e-9e89-e5374ac949fa'
 'd0aff0fd-f9bf-47ee-9925-71654effd36e'
 'ef084168-2342-4069-b1f6-148f98b1edd7'
 '8c01cec2-c7b1-46a3-8626-cfae34aa13d0'
 'd02ce1a7-cd45-43ea-8ed1-40128de8294b'
 '3828f066-1763-407a-85c7-c03716bca714'
 '33f70350-5bd4-4d43-96f4-6e09f85dbaba'
 '87fb42b0-8afc-453c-9549-f1a3ec9b50aa'
 '7a5e33b0-b588-40a0-b760-b13ae8303a4e'
 'b9d57b50-9b8e-443d-861a-e5856bd9e1cb'
 '95263334-f073-441f-825c-e1355b6584bd'
 'ed604d2b-55c7-41a5-ac19-78ebd9ea5cf1'
 '77fb2025-4198-4912-bcc5-9b252570c028']
Download and organization complete!


In [55]:
import os
import pandas as pd
import boto3

# Step 1: Load the CSV and filter for January 14, 2025
def get_new_session_ids(csv_file_path, date_to_filter):
    """
    Load the CSV file, filter rows by date, and return unique session IDs.

    Args:
        csv_file_path (str): Path to the CSV file.
        date_to_filter (str): The date to filter (format: 'YYYY-MM-DD').

    Returns:
        list: Unique session IDs for the filtered date.
    """
    # Load the CSV
    data = pd.read_csv(csv_file_path, dtype={"imei": "int64"})

    # Ensure 'created_at' is a datetime column
    data['created_at'] = pd.to_datetime(data['created_at'])

    # Filter rows for the specific date
    filtered_data = data[data['created_at'].dt.date == pd.Timestamp(date_to_filter).date()]

    # Extract and return unique session IDs
    return filtered_data['session_id'].unique()

# Step 2: Download and organize data from S3
def download_and_organize_new_data(session_ids, bucket_name, s3_folder_prefix, local_root_folder):
    """
    Download images from S3 for given session IDs and organize them into folders.

    Args:
        session_ids (list): List of session IDs to download.
        bucket_name (str): The S3 bucket name.
        s3_folder_prefix (str): The S3 folder prefix for images.
        local_root_folder (str): Local root folder to store the new data.
    """
    # Initialize S3 client
    s3 = boto3.client('s3')

    # Ensure the local root folder exists
    os.makedirs(local_root_folder, exist_ok=True)

    # Initialize paginator to handle S3 pagination
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder_prefix)

    # Download and organize files
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                file_key = obj['Key']

                # Skip folder or non-image files
                if file_key == s3_folder_prefix or not file_key.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                # Extract session_id from the file name
                file_name = os.path.basename(file_key)
                session_id = file_name.split('_INPUT')[0]

                # Process only for the filtered session IDs
                if session_id not in session_ids:
                    continue

                # Define local file path
                session_folder = os.path.join(local_root_folder, session_id)
                os.makedirs(session_folder, exist_ok=True)
                local_file_path = os.path.join(session_folder, file_name)

                # Skip if file already exists locally
                if os.path.exists(local_file_path):
                    print(f"File already exists, skipping: {local_file_path}")
                    continue

                # Download the file
                print(f"Downloading {file_key} to {local_file_path}")
                s3.download_file(bucket_name, file_key, local_file_path)

    print("Download and organization complete!")

In [ ]:
bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './test_images_error'  # Local folder for new data

# Step 1: Get session IDs for January 14, 2025
new_session_ids = ["1016a92d-da28-479d-afec-88839efd2ac6"]

# Step 2: Download and organize data for the filtered session IDs
download_and_organize_new_data(new_session_ids, bucket_name, s3_folder_prefix, local_root_folder)

In [44]:
import pandas as pd

# Load the data

file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_14thjan_dbdata.csv'  # Path to  CSV

df =pd.read_csv(file_path, dtype={"imei": "int64"})

df_before_100 = df.iloc[:100]
df_after_100 = df.iloc[100:]

# Convert the 'created_at' column for rows before 100
# df_before_100['created_at'] = pd.to_datetime(df_before_100['created_at'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

# Convert the 'created_at' column for rows after 100
df_after_100['created_at'] = pd.to_datetime(df_after_100['created_at'], errors='coerce', infer_datetime_format=True)

# Combine the two parts back into one DataFrame
df_cleaned = pd.concat([df_before_100, df_after_100], ignore_index=True)
df_cleaned['created_at'] = pd.to_datetime(df_cleaned['created_at'], errors='coerce', infer_datetime_format=True)

# Verify the conversion
print(df_cleaned.info())
print(df_cleaned[['created_at']].head(105))  # Display a few rows for verification

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          115 non-null    object                   
 1   api_key_id          115 non-null    int64                    
 2   imei                115 non-null    int64                    
 3   ai_screen_grading   115 non-null    object                   
 4   qc_screen_grading   115 non-null    object                   
 5   qc_screen_remark    75 non-null     object                   
 6   ai_housing_grading  115 non-null    object                   
 7   qc_housing_grading  115 non-null    object                   
 8   qc_housing_remark   71 non-null     object                   
 9   status              115 non-null    object                   
 10  active              115 non-null    bool                     
 11  created_at         

/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_90960/2377975432.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_after_100['created_at'] = pd.to_datetime(df_after_100['created_at'], errors='coerce', infer_datetime_format=True)
/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_90960/2377975432.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_100['created_at'] = pd.to_datetime(df_after_100['created_at'], errors='coerce', infer_datetime_format=True)
/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000g

In [45]:
# Ensure 'created_at' is in datetime format

# Filter rows where 'created_at' is January 14, 2025
df_cleaned[df_cleaned['created_at'].dt.date == pd.Timestamp('2025-01-14').date()]

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
100,a832c51a-9669-4eeb-aa43-b4fff74b0698,2,356179101057448,A2,C1,heavy scratches,A2,C1,"discoloration, very heavy back cover spots",QC-Evaluated,True,2025-01-14 09:26:13.355000+08:00,2025-01-14 09:28:24.034 +0800,NaN
101,7680de40-46fb-41c5-95d0-f1062254bd48,2,354626228115006,A2,A1,no scratches,C1,A3,dents,QC-Evaluated,True,2025-01-14 09:28:38.724000+08:00,2025-01-14 09:32:47.033 +0800,NaN
102,4fb94a16-fb78-4b9e-9e89-e5374ac949fa,2,354626222612206,A1,C1,heavy scratches,C1,A2,dents,QC-Evaluated,True,2025-01-14 09:34:12.054000+08:00,2025-01-14 09:37:12.099 +0800,NaN
103,d0aff0fd-f9bf-47ee-9925-71654effd36e,2,860366064824620,R2,R2,NaN,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-14 09:37:25.827000+08:00,2025-01-14 09:42:23.183 +0800,NaN
104,ef084168-2342-4069-b1f6-148f98b1edd7,2,867300056097591,C1,A1,no scratches,C1,A1,no scratches and dents,QC-Evaluated,True,2025-01-14 09:42:49.638000+08:00,2025-01-14 09:44:48.359 +0800,NaN
105,8c01cec2-c7b1-46a3-8626-cfae34aa13d0,2,357771778748199,A3,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-14 09:45:21.460000+08:00,2025-01-14 09:47:37.631 +0800,NaN
106,d02ce1a7-cd45-43ea-8ed1-40128de8294b,2,353134101147825,C1,C1,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-14 09:47:47.087000+08:00,2025-01-14 09:49:21.030 +0800,NaN
107,3828f066-1763-407a-85c7-c03716bca714,2,353499865835430,C1,A1,micro scratches,R2,A3,"back cover spots, micro scratches",QC-Evaluated,True,2025-01-14 09:49:28.981000+08:00,2025-01-14 09:53:20.716 +0800,NaN
108,33f70350-5bd4-4d43-96f4-6e09f85dbaba,2,869545048517456,A2,A1,micro scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-14 09:53:28.454000+08:00,2025-01-14 09:56:48.215 +0800,NaN
109,87fb42b0-8afc-453c-9549-f1a3ec9b50aa,2,352141104314782,A2,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-14 09:57:16.824000+08:00,2025-01-14 09:59:52.393 +0800,NaN


In [46]:
df_cleaned.to_csv('/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_14thjan_dbdata_datefix.csv', index=False)

## Analysing grading failure cases

In [2]:
import pandas as pd

In [3]:
full_data_path = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/testing_grading_accuracy/ai_session_202501201614.csv"
full_data = pd.read_csv(full_data_path)
full_data.head()

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,f6c9956b-6e2f-495b-a398-c96230f9721b,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:31:43.572 +0800,2024-12-17 15:31:43.572 +0800,NaN
1,447f4e42-07d8-4568-9e0a-9339c8bc62b9,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:32:42.154 +0800,2024-12-17 15:32:42.154 +0800,NaN
2,9dc13e75-b848-46e2-bc64-9bdf3ba87aac,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:45:56.018 +0800,2024-12-17 15:45:56.018 +0800,NaN
3,b664016d-dc6f-46b7-aecc-50c64a883fe7,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 16:00:53.444 +0800,2024-12-17 16:00:53.444 +0800,NaN
4,7f2117d9-37bb-4303-ab6c-6e5477ffa785,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 16:14:48.372 +0800,2024-12-17 16:14:48.372 +0800,NaN


In [6]:
full_data['status'].unique()

array([nan, 'Open', 'Grading_Failed', 'QC-Evaluated', 'Grading_Sucess',
       'Processing'], dtype=object)

In [9]:
# Filter the data where status is 'Grading_Failed'
grading_failed_data = full_data[full_data['status'] == 'Grading_Failed']

# Display the filtered data
grading_failed_data = grading_failed_data.reset_index(drop=True)
grading_failed_data = grading_failed_data[1:]
grading_failed_data = grading_failed_data.reset_index(drop=True)
grading_failed_data

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,ab5a3d05-970e-422f-a08c-cb766866160b,2,354208272100232,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 08:29:26.196 +0800,2025-01-17 08:34:22.050 +0800,NaN
1,0e6fb41a-68c8-4814-ac7b-15dd5af7dc48,2,354208272100232,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 09:09:11.188 +0800,2025-01-17 09:10:43.289 +0800,NaN
2,5999a507-45b2-4fac-a40c-53ce12d92d21,2,354208272100232,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 09:11:12.285 +0800,2025-01-17 09:13:59.177 +0800,NaN
3,5cd6c3f9-1979-4a8c-bcd0-562604ba884c,1,355786373017693,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 16:39:51.567 +0800,2025-01-17 16:40:45.955 +0800,NaN
4,f136bed5-5370-4037-a660-ac138c00c6df,2,359473642858969,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 09:36:27.704 +0800,2025-01-17 09:38:14.054 +0800,NaN
5,1e3b0761-9bc6-446e-b1b4-ec63f919dc70,1,355731672378651,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 13:00:09.738 +0800,2025-01-17 13:07:18.914 +0800,NaN
6,7a838871-522d-4616-b0dd-3afdafc0ce47,2,359473642858969,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 09:38:32.694 +0800,2025-01-17 09:39:45.681 +0800,NaN
7,b166653b-3e96-4287-a911-169d2e95ee94,1,867958042974114,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 12:39:27.422 +0800,2025-01-17 12:41:17.720 +0800,NaN
8,d41d5ed6-b15a-489d-9263-12335dd64ff6,1,355786373017693,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 16:34:34.995 +0800,2025-01-17 16:36:06.068 +0800,NaN
9,b02ce17d-c4ac-409b-9309-57ef2dada183,1,358863661231901,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-17 13:10:05.640 +0800,2025-01-17 13:11:31.668 +0800,NaN


In [10]:
unique_session_ids = grading_failed_data['session_id'].unique().tolist()
print(unique_session_ids)

['ab5a3d05-970e-422f-a08c-cb766866160b', '0e6fb41a-68c8-4814-ac7b-15dd5af7dc48', '5999a507-45b2-4fac-a40c-53ce12d92d21', '5cd6c3f9-1979-4a8c-bcd0-562604ba884c', 'f136bed5-5370-4037-a660-ac138c00c6df', '1e3b0761-9bc6-446e-b1b4-ec63f919dc70', '7a838871-522d-4616-b0dd-3afdafc0ce47', 'b166653b-3e96-4287-a911-169d2e95ee94', 'd41d5ed6-b15a-489d-9263-12335dd64ff6', 'b02ce17d-c4ac-409b-9309-57ef2dada183', 'f0648696-de3f-4bc7-b260-007411215bce', 'be820e03-056b-4b92-9cc5-05d565b6a3b4', 'dbb075ba-4a6b-4e2e-8dfe-088fe6e40aae', '1016a92d-da28-479d-afec-88839efd2ac6']


In [12]:
bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './grading_failure'  # Local folder for new data

# Step 1: Get session IDs for January 14, 2025
new_session_ids = unique_session_ids

# Step 2: Download and organize data for the filtered session IDs
download_and_organize_new_data(new_session_ids, bucket_name, s3_folder_prefix, local_root_folder)

Download and organization complete!


### Download data for failure 21 jan

In [59]:


bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './grading_failure_21Jan'  # Local folder for new data

# Step 1: Get session IDs for January 14, 2025
new_session_ids = ["84a7c7f5-4574-4e21-932f-fab5dcb32e36"]

# Step 2: Download and organize data for the filtered session IDs
download_and_organize_new_data(new_session_ids, bucket_name, s3_folder_prefix, local_root_folder)

Download and organization complete!


In [60]:
failure_ids =["2b2179d1-7b0b-4636-9771-7e6a83f023b9", "5e959ac1-5bd4-458a-9f29-792a12eb8e7d", "b88b66c2-b3e2-46ea-b1a3-d1c2882b6558", 
              "4939d0b4-e636-417e-bedb-f86dd88852cb", "58bf9723-c5e9-45b5-82a4-6fb62427a673", "a3919105-311e-43d0-96a9-ea57081a8943",
              "67e05fdf-2de6-420d-99b3-58f2e18fe94e", "c16423ad-ea32-4ef0-84ca-d808d196abea"]

bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './grading_failure_21Jan'  # Local folder for new data

download_and_organize_new_data(failure_ids, bucket_name, s3_folder_prefix, local_root_folder)

Download and organization complete!


# Cleaning and Merginfg new Data

## Filter data after 14th Jan

In [17]:
# Ensure 'created_at' is in datetime format
full_data['created_at'] = pd.to_datetime(full_data['created_at'], errors='coerce', infer_datetime_format=True)

# Localize the comparison timestamp to the same timezone as 'created_at'
comparison_timestamp = pd.Timestamp('2025-01-15').tz_localize(full_data['created_at'].dt.tz)

# Filter rows where 'created_at' is after January 14, 2025
filtered_data = full_data[full_data['created_at'] > comparison_timestamp]

filtered_data = filtered_data.reset_index(drop=True)

# Display the filtered data
filtered_data

/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_92801/83986478.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  full_data['created_at'] = pd.to_datetime(full_data['created_at'], errors='coerce', infer_datetime_format=True)


,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,644478e0-8a95-4b55-ba90-ebae66996ac4,2,357217092920148,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-16 09:43:24.617000+08:00,2025-01-16 09:43:24.617 +0800,NaN
1,f474b93e-9a0b-4d03-abc9-2ca60c4248b2,2,354915111640323,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-15 09:36:18.448000+08:00,2025-01-15 09:36:18.448 +0800,NaN
2,d8aab13a-6de2-4fe6-ae2b-43e1cf71c227,2,351232995434960,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-16 09:10:42.507000+08:00,2025-01-16 09:10:42.507 +0800,NaN
3,5a928512-0fe4-45de-b955-64d8e4f6ee91,2,355705442472192,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 09:57:21.060000+08:00,2025-01-15 09:59:23.547 +0800,NaN
4,6fdc0367-6086-41bd-88e2-51ab2735f77b,2,354915111640323,A2,A1,no scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-15 09:36:18.501000+08:00,2025-01-15 09:40:14.739 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,65994e62-f8db-4d38-b411-f4f95a6eeece,2,358819100800201,A1,A1,NaN,A3,R2,"cracked, very heavy back cover spots",QC-Evaluated,True,2025-01-20 14:15:02.282000+08:00,2025-01-20 14:17:07.517 +0800,NaN
232,b823eb3e-61c9-42e7-8b1e-e7e9cd545d37,2,353305094592421,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-20 14:31:28.015000+08:00,2025-01-20 14:31:28.015 +0800,NaN
233,8c040446-650c-49fe-b93f-8ce0fd2c4f34,2,353305094592421,R2,R2,NaN,R2,C1,"very heavy discoloration, dents",QC-Evaluated,True,2025-01-20 14:31:28.066000+08:00,2025-01-20 14:33:14.608 +0800,NaN
234,9e99a4dc-7e7c-447f-a1fd-b58eccbace56,2,359943091018422,A1,C1,heavy scratches,A2,C1,"dents, very heavy back cover spots",QC-Evaluated,True,2025-01-20 14:25:27.789000+08:00,2025-01-20 14:28:25.407 +0800,NaN


### Drop NAN qc grades rows

In [19]:
filtered_non_nan_data = filtered_data.dropna(subset=['qc_screen_grading', 'qc_housing_grading'], how='any')
filtered_non_nan_data = filtered_non_nan_data.reset_index(drop=True)
filtered_non_nan_data

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,5a928512-0fe4-45de-b955-64d8e4f6ee91,2,355705442472192,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 09:57:21.060000+08:00,2025-01-15 09:59:23.547 +0800,NaN
1,6fdc0367-6086-41bd-88e2-51ab2735f77b,2,354915111640323,A2,A1,no scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-15 09:36:18.501000+08:00,2025-01-15 09:40:14.739 +0800,NaN
2,0581e58b-24e0-45ca-ac79-f8d3df1bf1dc,2,359943096908171,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:40:31.689000+08:00,2025-01-15 09:42:14.138 +0800,NaN
3,df16c688-0a8c-4bf1-b660-4dca8f1ce7ce,2,352830111101972,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 10:09:23.811000+08:00,2025-01-15 10:11:53.649 +0800,NaN
4,ca67c99a-5700-4b94-aa35-3f6cf6785176,2,354822094701652,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:42:20.144000+08:00,2025-01-15 09:44:03.806 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,79a5fd99-917f-4d4d-be1a-62972b1ae148,2,359943098981713,R2,R2,NaN,A1,C1,very heavy discoloration,QC-Evaluated,True,2025-01-20 14:28:49.214000+08:00,2025-01-20 14:31:22.912 +0800,NaN
91,0eb1cf27-3b1b-4090-9e10-db7a0b0fc1c8,2,351591115350681,A1,A1,NaN,A1,A3,"heavy back cover spots, micro scratches",QC-Evaluated,True,2025-01-20 14:20:42.803000+08:00,2025-01-20 14:25:19.234 +0800,NaN
92,65994e62-f8db-4d38-b411-f4f95a6eeece,2,358819100800201,A1,A1,NaN,A3,R2,"cracked, very heavy back cover spots",QC-Evaluated,True,2025-01-20 14:15:02.282000+08:00,2025-01-20 14:17:07.517 +0800,NaN
93,8c040446-650c-49fe-b93f-8ce0fd2c4f34,2,353305094592421,R2,R2,NaN,R2,C1,"very heavy discoloration, dents",QC-Evaluated,True,2025-01-20 14:31:28.066000+08:00,2025-01-20 14:33:14.608 +0800,NaN


In [20]:
filtered_non_nan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          95 non-null     object                   
 1   api_key_id          95 non-null     int64                    
 2   imei                95 non-null     object                   
 3   ai_screen_grading   95 non-null     object                   
 4   qc_screen_grading   95 non-null     object                   
 5   qc_screen_remark    50 non-null     object                   
 6   ai_housing_grading  95 non-null     object                   
 7   qc_housing_grading  95 non-null     object                   
 8   qc_housing_remark   60 non-null     object                   
 9   status              95 non-null     object                   
 10  active              95 non-null     bool                     
 11  created_at          9

In [28]:
old_data_path = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_14thjan_dbdata.csv"
old_data = pd.read_csv(old_data_path)
old_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          115 non-null    object 
 1   api_key_id          115 non-null    int64  
 2   imei                115 non-null    float64
 3   ai_screen_grading   115 non-null    object 
 4   qc_screen_grading   115 non-null    object 
 5   qc_screen_remark    75 non-null     object 
 6   ai_housing_grading  115 non-null    object 
 7   qc_housing_grading  115 non-null    object 
 8   qc_housing_remark   71 non-null     object 
 9   status              115 non-null    object 
 10  active              115 non-null    bool   
 11  created_at          115 non-null    object 
 12  updated_at          115 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 11.9+ KB


In [24]:
old_data['imei'] = old_data['imei'].astype('int64')
old_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          115 non-null    object 
 1   api_key_id          115 non-null    int64  
 2   imei                115 non-null    int64  
 3   ai_screen_grading   115 non-null    object 
 4   qc_screen_grading   115 non-null    object 
 5   qc_screen_remark    75 non-null     object 
 6   ai_housing_grading  115 non-null    object 
 7   qc_housing_grading  115 non-null    object 
 8   qc_housing_remark   71 non-null     object 
 9   status              115 non-null    object 
 10  active              115 non-null    bool   
 11  created_at          115 non-null    object 
 12  updated_at          115 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 11.9+ KB


In [29]:
old_data[99:]

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
99,d5c9c96f-243f-4b49-b076-3460a6252eed,2,3.564851e+14,A1,A1,NaN,C1,A3,"back cover spots, paint peelings",QC-Evaluated,True,2025-01-10 11:17:38.311000+08:00,2025-01-10 11:20:26.616 +0800,NaN
100,a832c51a-9669-4eeb-aa43-b4fff74b0698,2,3.561791e+14,A2,C1,heavy scratches,A2,C1,"discoloration, very heavy back cover spots",QC-Evaluated,True,2025-01-14 09:26:13.355 +0800,2025-01-14 09:28:24.034 +0800,NaN
101,7680de40-46fb-41c5-95d0-f1062254bd48,2,3.546262e+14,A2,A1,no scratches,C1,A3,dents,QC-Evaluated,True,2025-01-14 09:28:38.724 +0800,2025-01-14 09:32:47.033 +0800,NaN
102,4fb94a16-fb78-4b9e-9e89-e5374ac949fa,2,3.546262e+14,A1,C1,heavy scratches,C1,A2,dents,QC-Evaluated,True,2025-01-14 09:34:12.054 +0800,2025-01-14 09:37:12.099 +0800,NaN
103,d0aff0fd-f9bf-47ee-9925-71654effd36e,2,8.603661e+14,R2,R2,NaN,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-14 09:37:25.827 +0800,2025-01-14 09:42:23.183 +0800,NaN
104,ef084168-2342-4069-b1f6-148f98b1edd7,2,8.673001e+14,C1,A1,no scratches,C1,A1,no scratches and dents,QC-Evaluated,True,2025-01-14 09:42:49.638 +0800,2025-01-14 09:44:48.359 +0800,NaN
105,8c01cec2-c7b1-46a3-8626-cfae34aa13d0,2,3.577718e+14,A3,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-14 09:45:21.460 +0800,2025-01-14 09:47:37.631 +0800,NaN
106,d02ce1a7-cd45-43ea-8ed1-40128de8294b,2,3.531341e+14,C1,C1,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-14 09:47:47.087 +0800,2025-01-14 09:49:21.030 +0800,NaN
107,3828f066-1763-407a-85c7-c03716bca714,2,3.534999e+14,C1,A1,micro scratches,R2,A3,"back cover spots, micro scratches",QC-Evaluated,True,2025-01-14 09:49:28.981 +0800,2025-01-14 09:53:20.716 +0800,NaN
108,33f70350-5bd4-4d43-96f4-6e09f85dbaba,2,8.695450e+14,A2,A1,micro scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-14 09:53:28.454 +0800,2025-01-14 09:56:48.215 +0800,NaN


In [34]:
# Convert the 'created_at' column in old_data to datetime format with the same timezone as filtered_data
# old_data['created_at'] = pd.to_datetime(old_data['created_at'], errors='coerce', infer_datetime_format=True).dt.tz_convert('UTC+08:00')

def preprocess_date(data):
    """
    Preprocess the data by converting 'created_at' to datetime and handling NaT values.

    Args:
        data (pd.DataFrame): The input data to preprocess.

    Returns:
        pd.DataFrame: The preprocessed data.
    """
    # Convert 'created_at' to datetime with timezone conversion
    data['created_at'] = pd.to_datetime(data['created_at'], errors='coerce', infer_datetime_format=True).dt.tz_convert('UTC+08:00')
    
    # Handle NaT values by filling with a specific date or dropping them
    data['created_at'].fillna(pd.Timestamp('2025-01-01').tz_localize('UTC+08:00'), inplace=True)
    
    return data

# Apply the preprocess method to old_data
# old_data = preprocess_data(old_data)
# Verify the conversion

old_data = preprocess_date(old_data)

print(old_data.info())
old_data[99:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          115 non-null    object                   
 1   api_key_id          115 non-null    int64                    
 2   imei                115 non-null    float64                  
 3   ai_screen_grading   115 non-null    object                   
 4   qc_screen_grading   115 non-null    object                   
 5   qc_screen_remark    75 non-null     object                   
 6   ai_housing_grading  115 non-null    object                   
 7   qc_housing_grading  115 non-null    object                   
 8   qc_housing_remark   71 non-null     object                   
 9   status              115 non-null    object                   
 10  active              115 non-null    bool                     
 11  created_at         

/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_92801/1924730718.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data['created_at'] = pd.to_datetime(data['created_at'], errors='coerce', infer_datetime_format=True).dt.tz_convert('UTC+08:00')
/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_92801/1924730718.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(valu

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
99,d5c9c96f-243f-4b49-b076-3460a6252eed,2,3.564851e+14,A1,A1,NaN,C1,A3,"back cover spots, paint peelings",QC-Evaluated,True,2025-01-10 11:17:38.311000+08:00,2025-01-10 11:20:26.616 +0800,NaN
100,a832c51a-9669-4eeb-aa43-b4fff74b0698,2,3.561791e+14,A2,C1,heavy scratches,A2,C1,"discoloration, very heavy back cover spots",QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:28:24.034 +0800,NaN
101,7680de40-46fb-41c5-95d0-f1062254bd48,2,3.546262e+14,A2,A1,no scratches,C1,A3,dents,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:32:47.033 +0800,NaN
102,4fb94a16-fb78-4b9e-9e89-e5374ac949fa,2,3.546262e+14,A1,C1,heavy scratches,C1,A2,dents,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:37:12.099 +0800,NaN
103,d0aff0fd-f9bf-47ee-9925-71654effd36e,2,8.603661e+14,R2,R2,NaN,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:42:23.183 +0800,NaN
104,ef084168-2342-4069-b1f6-148f98b1edd7,2,8.673001e+14,C1,A1,no scratches,C1,A1,no scratches and dents,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:44:48.359 +0800,NaN
105,8c01cec2-c7b1-46a3-8626-cfae34aa13d0,2,3.577718e+14,A3,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:47:37.631 +0800,NaN
106,d02ce1a7-cd45-43ea-8ed1-40128de8294b,2,3.531341e+14,C1,C1,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:49:21.030 +0800,NaN
107,3828f066-1763-407a-85c7-c03716bca714,2,3.534999e+14,C1,A1,micro scratches,R2,A3,"back cover spots, micro scratches",QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:53:20.716 +0800,NaN
108,33f70350-5bd4-4d43-96f4-6e09f85dbaba,2,8.695450e+14,A2,A1,micro scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:56:48.215 +0800,NaN


### Drop rows with api key 1

In [38]:
unique_api_keys = filtered_non_nan_data['api_key_id'].unique()
print(unique_api_keys)

[2 1]


In [46]:
filtered_non_nan_data_api_key_2 = filtered_non_nan_data[filtered_non_nan_data['api_key_id'] == 2]
filtered_non_nan_data_api_key_2 = filtered_non_nan_data_api_key_2.reset_index(drop=True)
filtered_non_nan_data_api_key_2

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,5a928512-0fe4-45de-b955-64d8e4f6ee91,2,355705442472192,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 09:57:21.060000+08:00,2025-01-15 09:59:23.547 +0800,NaN
1,6fdc0367-6086-41bd-88e2-51ab2735f77b,2,354915111640323,A2,A1,no scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-15 09:36:18.501000+08:00,2025-01-15 09:40:14.739 +0800,NaN
2,0581e58b-24e0-45ca-ac79-f8d3df1bf1dc,2,359943096908171,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:40:31.689000+08:00,2025-01-15 09:42:14.138 +0800,NaN
3,df16c688-0a8c-4bf1-b660-4dca8f1ce7ce,2,352830111101972,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 10:09:23.811000+08:00,2025-01-15 10:11:53.649 +0800,NaN
4,ca67c99a-5700-4b94-aa35-3f6cf6785176,2,354822094701652,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:42:20.144000+08:00,2025-01-15 09:44:03.806 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,79a5fd99-917f-4d4d-be1a-62972b1ae148,2,359943098981713,R2,R2,NaN,A1,C1,very heavy discoloration,QC-Evaluated,True,2025-01-20 14:28:49.214000+08:00,2025-01-20 14:31:22.912 +0800,NaN
72,0eb1cf27-3b1b-4090-9e10-db7a0b0fc1c8,2,351591115350681,A1,A1,NaN,A1,A3,"heavy back cover spots, micro scratches",QC-Evaluated,True,2025-01-20 14:20:42.803000+08:00,2025-01-20 14:25:19.234 +0800,NaN
73,65994e62-f8db-4d38-b411-f4f95a6eeece,2,358819100800201,A1,A1,NaN,A3,R2,"cracked, very heavy back cover spots",QC-Evaluated,True,2025-01-20 14:15:02.282000+08:00,2025-01-20 14:17:07.517 +0800,NaN
74,8c040446-650c-49fe-b93f-8ce0fd2c4f34,2,353305094592421,R2,R2,NaN,R2,C1,"very heavy discoloration, dents",QC-Evaluated,True,2025-01-20 14:31:28.066000+08:00,2025-01-20 14:33:14.608 +0800,NaN


### Merge with old

In [47]:
# Merge old data with the latest non-nan filtered data
merged_data = pd.concat([old_data, filtered_non_nan_data_api_key_2], ignore_index=True)

# Verify the merge
print(merged_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          191 non-null    object                   
 1   api_key_id          191 non-null    int64                    
 2   imei                191 non-null    object                   
 3   ai_screen_grading   191 non-null    object                   
 4   qc_screen_grading   191 non-null    object                   
 5   qc_screen_remark    123 non-null    object                   
 6   ai_housing_grading  191 non-null    object                   
 7   qc_housing_grading  191 non-null    object                   
 8   qc_housing_remark   127 non-null    object                   
 9   status              191 non-null    object                   
 10  active              191 non-null    bool                     
 11  created_at         

In [54]:
# Ensure 'created_at' is in datetime format
# merged_data['created_at'] = pd.to_datetime(merged_data['created_at'], errors='coerce', infer_datetime_format=True)

# Localize the comparison timestamp to the same timezone as 'created_at'
comparison_timestamp = pd.Timestamp('2025-01-14').tz_localize('UTC+08:00')

# Filter rows where 'created_at' is after January 14, 2025
filtered_merged_data = merged_data[merged_data['created_at'] > comparison_timestamp]

filtered_merged_data = filtered_merged_data.reset_index(drop=True)
filtered_merged_data

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,5a928512-0fe4-45de-b955-64d8e4f6ee91,2,355705442472192,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 09:57:21.060000+08:00,2025-01-15 09:59:23.547 +0800,NaN
1,6fdc0367-6086-41bd-88e2-51ab2735f77b,2,354915111640323,A2,A1,no scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-15 09:36:18.501000+08:00,2025-01-15 09:40:14.739 +0800,NaN
2,0581e58b-24e0-45ca-ac79-f8d3df1bf1dc,2,359943096908171,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:40:31.689000+08:00,2025-01-15 09:42:14.138 +0800,NaN
3,df16c688-0a8c-4bf1-b660-4dca8f1ce7ce,2,352830111101972,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-15 10:09:23.811000+08:00,2025-01-15 10:11:53.649 +0800,NaN
4,ca67c99a-5700-4b94-aa35-3f6cf6785176,2,354822094701652,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-15 09:42:20.144000+08:00,2025-01-15 09:44:03.806 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,79a5fd99-917f-4d4d-be1a-62972b1ae148,2,359943098981713,R2,R2,NaN,A1,C1,very heavy discoloration,QC-Evaluated,True,2025-01-20 14:28:49.214000+08:00,2025-01-20 14:31:22.912 +0800,NaN
72,0eb1cf27-3b1b-4090-9e10-db7a0b0fc1c8,2,351591115350681,A1,A1,NaN,A1,A3,"heavy back cover spots, micro scratches",QC-Evaluated,True,2025-01-20 14:20:42.803000+08:00,2025-01-20 14:25:19.234 +0800,NaN
73,65994e62-f8db-4d38-b411-f4f95a6eeece,2,358819100800201,A1,A1,NaN,A3,R2,"cracked, very heavy back cover spots",QC-Evaluated,True,2025-01-20 14:15:02.282000+08:00,2025-01-20 14:17:07.517 +0800,NaN
74,8c040446-650c-49fe-b93f-8ce0fd2c4f34,2,353305094592421,R2,R2,NaN,R2,C1,"very heavy discoloration, dents",QC-Evaluated,True,2025-01-20 14:31:28.066000+08:00,2025-01-20 14:33:14.608 +0800,NaN


In [58]:
merged_data.to_csv('/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/data_till_20Jan.csv', index=False)

In [63]:
merged_data
false_positive_r2 = merged_data[(merged_data['ai_housing_grading'] == 'R2') & (merged_data['qc_housing_grading'] != 'R2')]
false_positive_r2

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
107,3828f066-1763-407a-85c7-c03716bca714,2,353499865835430.0,C1,A1,micro scratches,R2,A3,"back cover spots, micro scratches",QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:53:20.716 +0800,NaN
108,33f70350-5bd4-4d43-96f4-6e09f85dbaba,2,869545048517456.0,A2,A1,micro scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 09:56:48.215 +0800,NaN
113,ed604d2b-55c7-41a5-ac19-78ebd9ea5cf1,2,358625332457730.0,A3,A1,micro scratches,R2,A1,no scratches and dents,QC-Evaluated,True,2025-01-01 00:00:00+08:00,2025-01-14 10:13:52.206 +0800,NaN
116,6fdc0367-6086-41bd-88e2-51ab2735f77b,2,354915111640323,A2,A1,no scratches,R2,A3,heavy back cover spots,QC-Evaluated,True,2025-01-15 09:36:18.501000+08:00,2025-01-15 09:40:14.739 +0800,NaN
121,956f3983-2338-48e4-88f7-21ca3b58ded6,2,353305097943373,R2,R2,NaN,R2,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-15 09:44:09.851000+08:00,2025-01-15 09:46:14.839 +0800,NaN
122,3206942a-1d95-4487-bd98-d81bae2b9a44,2,353305096377573,A3,A3,NaN,R2,C1,"major dents, discoloration, heavy scratches",QC-Evaluated,True,2025-01-15 09:46:22.961000+08:00,2025-01-15 09:49:50.516 +0800,NaN
127,35ec2c08-3855-4010-a4bc-e3d6e8e4b494,2,863477061736621,A2,A1,no scratches,R2,A1,no scratches,QC-Evaluated,True,2025-01-16 09:13:39.513000+08:00,2025-01-16 09:14:54.974 +0800,NaN
139,82100818-e094-498c-9a26-480c00e36bd9,2,864540059242107,R2,A1,no scratches,R2,A1,no scratches,QC-Evaluated,True,2025-01-16 09:44:25.913000+08:00,2025-01-16 09:45:29.600 +0800,NaN
161,e0053b14-d3fd-44e6-a256-5d72df464577,2,353305090066131,R2,R2,NaN,R2,C1,very heavy discoloration,QC-Evaluated,True,2025-01-20 14:33:22.894000+08:00,2025-01-20 14:35:04.150 +0800,NaN
162,dc9a3e57-dcb2-4c16-9d7a-3b997d2a4bac,2,356916093904750,A2,R2,cracked,R2,C1,major dents,QC-Evaluated,True,2025-01-20 14:06:21.805000+08:00,2025-01-20 14:07:52.232 +0800,NaN


### Downloading images against new session_ids

In [53]:
session_ids_after_14 = filtered_merged_data['session_id'].unique().tolist()

In [56]:
bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './session_ids_after_14'  # Local folder for new data

# Step 1: Get session IDs for January 14, 2025

# Step 2: Download and organize data for the filtered session IDs
download_and_organize_new_data(session_ids_after_14, bucket_name, s3_folder_prefix, local_root_folder)

Download and organization complete!


## Getting 22 jan data with 21 data

### load full csv

In [64]:
full_data_22jan = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/ai_session_202501221010.csv"
full_data_22jan = pd.read_csv(full_data_22jan)
full_data_22jan

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,f6c9956b-6e2f-495b-a398-c96230f9721b,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:31:43.572 +0800,2024-12-17 15:31:43.572 +0800,NaN
1,447f4e42-07d8-4568-9e0a-9339c8bc62b9,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:32:42.154 +0800,2024-12-17 15:32:42.154 +0800,NaN
2,9dc13e75-b848-46e2-bc64-9bdf3ba87aac,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 15:45:56.018 +0800,2024-12-17 15:45:56.018 +0800,NaN
3,b664016d-dc6f-46b7-aecc-50c64a883fe7,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 16:00:53.444 +0800,2024-12-17 16:00:53.444 +0800,NaN
4,7f2117d9-37bb-4303-ab6c-6e5477ffa785,1,2123456789012666345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2024-12-17 16:14:48.372 +0800,2024-12-17 16:14:48.372 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,14d68947-9e38-464e-b7dd-a7bb699ee897,2,359322445530731,A2,A1,no scratches,A2,A3,heavy back cover spot & discolouration,QC-Evaluated,True,2025-01-22 09:18:39.622 +0800,2025-01-22 09:20:29.551 +0800,NaN
773,0854b9fe-d31c-4817-a6d9-76f458efb0d5,2,353843101298616,A1,A1,NaN,A3,R2,"cracked, heavy scratches on bezel",QC-Evaluated,True,2025-01-22 09:19:09.525 +0800,2025-01-22 09:21:30.980 +0800,NaN
774,a802a55c-c6a8-4372-baa1-3e9c022b9376,2,353499863940372,A1,C1,heavy scratches,A2,C1,"very heavy back cover spots, dents",QC-Evaluated,True,2025-01-22 09:41:42.310 +0800,2025-01-22 09:43:10.850 +0800,NaN
775,37e961cd-5fd5-48b1-ac82-1d417f4d4db9,2,350145450452643,A1,A1,NaN,R2,A3,dents,QC-Evaluated,True,2025-01-22 09:43:24.869 +0800,2025-01-22 09:45:19.413 +0800,NaN


### filter with timestamp

In [65]:
# Ensure 'created_at' is in datetime format
full_data_22jan['created_at'] = pd.to_datetime(full_data_22jan['created_at'], errors='coerce', infer_datetime_format=True)

# Localize the comparison timestamp to the same timezone as 'created_at'
comparison_timestamp = pd.Timestamp('2025-01-21').tz_localize(full_data_22jan['created_at'].dt.tz)

# Filter rows where 'created_at' is after January 14, 2025
filtered_data = full_data_22jan[full_data_22jan['created_at'] > comparison_timestamp]

filtered_data = filtered_data.reset_index(drop=True)

# Display the filtered data
filtered_data

/var/folders/xx/x0vg36v535n5nlq2ts49kgsc0000gn/T/ipykernel_92801/2294530771.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  full_data_22jan['created_at'] = pd.to_datetime(full_data_22jan['created_at'], errors='coerce', infer_datetime_format=True)


,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,84a7c7f5-4574-4e21-932f-fab5dcb32e36,2,354626190782940,C1,NaN,NaN,A2,NaN,NaN,Grading_Sucess,True,2025-01-21 09:22:07.885000+08:00,2025-01-21 10:21:35.245 +0800,NaN
1,2b2179d1-7b0b-4636-9771-7e6a83f023b9,2,350635581088779,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-21 09:23:45.043000+08:00,2025-01-21 09:23:45.043 +0800,NaN
2,de825d71-bd5d-41d9-87b4-69d369ef6de2,2,354009841346060,A1,A1,NaN,A2,A3,"paint peelings, scratches on bezel",QC-Evaluated,True,2025-01-21 14:44:55.392000+08:00,2025-01-21 14:48:09.626 +0800,NaN
3,03cfa1bb-ee14-4bd3-9be9-995f406499f5,2,350635581088779,A2,NaN,NaN,A3,NaN,NaN,Grading_Sucess,True,2025-01-21 14:20:32.789000+08:00,2025-01-21 14:21:35.273 +0800,NaN
4,5e959ac1-5bd4-458a-9f29-792a12eb8e7d,2,355224388337757,NaN,NaN,NaN,NaN,NaN,NaN,Grading_Failed,True,2025-01-21 09:27:59.803000+08:00,2025-01-21 09:29:34.475 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,14d68947-9e38-464e-b7dd-a7bb699ee897,2,359322445530731,A2,A1,no scratches,A2,A3,heavy back cover spot & discolouration,QC-Evaluated,True,2025-01-22 09:18:39.622000+08:00,2025-01-22 09:20:29.551 +0800,NaN
88,0854b9fe-d31c-4817-a6d9-76f458efb0d5,2,353843101298616,A1,A1,NaN,A3,R2,"cracked, heavy scratches on bezel",QC-Evaluated,True,2025-01-22 09:19:09.525000+08:00,2025-01-22 09:21:30.980 +0800,NaN
89,a802a55c-c6a8-4372-baa1-3e9c022b9376,2,353499863940372,A1,C1,heavy scratches,A2,C1,"very heavy back cover spots, dents",QC-Evaluated,True,2025-01-22 09:41:42.310000+08:00,2025-01-22 09:43:10.850 +0800,NaN
90,37e961cd-5fd5-48b1-ac82-1d417f4d4db9,2,350145450452643,A1,A1,NaN,R2,A3,dents,QC-Evaluated,True,2025-01-22 09:43:24.869000+08:00,2025-01-22 09:45:19.413 +0800,NaN


### drop nan qc rows

In [66]:
filtered_non_nan_data = filtered_data.dropna(subset=['qc_screen_grading', 'qc_housing_grading'], how='any')
filtered_non_nan_data = filtered_non_nan_data.reset_index(drop=True)
filtered_non_nan_data

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,de825d71-bd5d-41d9-87b4-69d369ef6de2,2,354009841346060,A1,A1,NaN,A2,A3,"paint peelings, scratches on bezel",QC-Evaluated,True,2025-01-21 14:44:55.392000+08:00,2025-01-21 14:48:09.626 +0800,NaN
1,afd4349a-1b92-4bae-930f-1c120f8b15c1,2,357293092731850,A1,C1,heavy scratches,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:59:14.310000+08:00,2025-01-21 15:01:19.216 +0800,NaN
2,9744b947-f7aa-43a1-a7cf-1c4ef5d63781,2,352027111360258,A1,A1,NaN,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:29:53.649000+08:00,2025-01-21 14:32:00.402 +0800,NaN
3,fb9633e5-4a93-4457-b4f9-d783135fd537,2,351591110249300,C1,A1,no scratches,A2,C1,very heavy back cover spot,QC-Evaluated,True,2025-01-21 14:49:02+08:00,2025-01-21 14:50:21.097 +0800,NaN
4,94a7090f-9e22-4e1f-bd7f-f608df7c4dec,2,353418794348570,A1,A1,NaN,A1,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:32:15.348000+08:00,2025-01-21 14:34:31.063 +0800,NaN
5,c32e0927-e8aa-431d-9be2-5f657013c9d4,2,353904105654590,A1,A1,NaN,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:49:19.543000+08:00,2025-01-21 14:50:54.352 +0800,NaN
6,75fd3936-aeb3-4a43-a7a7-8acc0290a923,2,356716111111168,A1,C1,heavy scratches,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:34:52.270000+08:00,2025-01-21 14:37:33.967 +0800,NaN
7,1975a992-e98a-4c3f-88be-c9157ec76ff2,2,352141100694914,R2,R2,NaN,A3,C1,very heavy discoloration,QC-Evaluated,True,2025-01-21 15:03:15.835000+08:00,2025-01-21 15:04:42.322 +0800,NaN
8,69468147-99d5-4dea-8af1-272b66adee7a,1,353017110102586,A1,A1,NaN,A2,A2,NaN,QC-Evaluated,True,2025-01-21 11:12:06.216000+08:00,2025-01-21 11:14:01.313 +0800,NaN
9,f2468a34-4765-42c7-a7f9-c663f2906dfc,2,355066961222728,A1,A3,minor scratches,A1,C1,"dents, heavy discoloration, back cover spots",QC-Evaluated,True,2025-01-21 14:37:46.745000+08:00,2025-01-21 14:39:49.523 +0800,NaN


### filter for api key 2

In [67]:
filtered_non_nan_data_api_key_2 = filtered_non_nan_data[filtered_non_nan_data['api_key_id'] == 2]
filtered_non_nan_data_api_key_2 = filtered_non_nan_data_api_key_2.reset_index(drop=True)
filtered_non_nan_data_api_key_2

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,de825d71-bd5d-41d9-87b4-69d369ef6de2,2,354009841346060,A1,A1,NaN,A2,A3,"paint peelings, scratches on bezel",QC-Evaluated,True,2025-01-21 14:44:55.392000+08:00,2025-01-21 14:48:09.626 +0800,NaN
1,afd4349a-1b92-4bae-930f-1c120f8b15c1,2,357293092731850,A1,C1,heavy scratches,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:59:14.310000+08:00,2025-01-21 15:01:19.216 +0800,NaN
2,9744b947-f7aa-43a1-a7cf-1c4ef5d63781,2,352027111360258,A1,A1,NaN,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:29:53.649000+08:00,2025-01-21 14:32:00.402 +0800,NaN
3,fb9633e5-4a93-4457-b4f9-d783135fd537,2,351591110249300,C1,A1,no scratches,A2,C1,very heavy back cover spot,QC-Evaluated,True,2025-01-21 14:49:02+08:00,2025-01-21 14:50:21.097 +0800,NaN
4,94a7090f-9e22-4e1f-bd7f-f608df7c4dec,2,353418794348570,A1,A1,NaN,A1,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:32:15.348000+08:00,2025-01-21 14:34:31.063 +0800,NaN
5,c32e0927-e8aa-431d-9be2-5f657013c9d4,2,353904105654590,A1,A1,NaN,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:49:19.543000+08:00,2025-01-21 14:50:54.352 +0800,NaN
6,75fd3936-aeb3-4a43-a7a7-8acc0290a923,2,356716111111168,A1,C1,heavy scratches,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:34:52.270000+08:00,2025-01-21 14:37:33.967 +0800,NaN
7,1975a992-e98a-4c3f-88be-c9157ec76ff2,2,352141100694914,R2,R2,NaN,A3,C1,very heavy discoloration,QC-Evaluated,True,2025-01-21 15:03:15.835000+08:00,2025-01-21 15:04:42.322 +0800,NaN
8,f2468a34-4765-42c7-a7f9-c663f2906dfc,2,355066961222728,A1,A3,minor scratches,A1,C1,"dents, heavy discoloration, back cover spots",QC-Evaluated,True,2025-01-21 14:37:46.745000+08:00,2025-01-21 14:39:49.523 +0800,NaN
9,c26dfd36-c626-4601-b270-1a42b004d2de,2,350145451064280,A1,A1,NaN,A3,A2,"paint peelings, dings",QC-Evaluated,True,2025-01-21 14:54:58.607000+08:00,2025-01-21 14:58:58.055 +0800,NaN


In [68]:
filtered_non_nan_data_api_key_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          55 non-null     object                   
 1   api_key_id          55 non-null     int64                    
 2   imei                55 non-null     object                   
 3   ai_screen_grading   55 non-null     object                   
 4   qc_screen_grading   55 non-null     object                   
 5   qc_screen_remark    20 non-null     object                   
 6   ai_housing_grading  55 non-null     object                   
 7   qc_housing_grading  55 non-null     object                   
 8   qc_housing_remark   40 non-null     object                   
 9   status              55 non-null     object                   
 10  active              55 non-null     bool                     
 11  created_at          5

In [69]:
# Convert the 'imei' column to int64
filtered_non_nan_data_api_key_2['imei'] = filtered_non_nan_data_api_key_2['imei'].astype('int64')

# Verify the conversion
filtered_non_nan_data_api_key_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          55 non-null     object                   
 1   api_key_id          55 non-null     int64                    
 2   imei                55 non-null     int64                    
 3   ai_screen_grading   55 non-null     object                   
 4   qc_screen_grading   55 non-null     object                   
 5   qc_screen_remark    20 non-null     object                   
 6   ai_housing_grading  55 non-null     object                   
 7   qc_housing_grading  55 non-null     object                   
 8   qc_housing_remark   40 non-null     object                   
 9   status              55 non-null     object                   
 10  active              55 non-null     bool                     
 11  created_at          5

### Merge

In [74]:
merged_data_b421 = merged_data.copy()
merged_data_b421['imei'] = merged_data_b421['imei'].astype('int64')

merged_data_b421.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          191 non-null    object                   
 1   api_key_id          191 non-null    int64                    
 2   imei                191 non-null    int64                    
 3   ai_screen_grading   191 non-null    object                   
 4   qc_screen_grading   191 non-null    object                   
 5   qc_screen_remark    123 non-null    object                   
 6   ai_housing_grading  191 non-null    object                   
 7   qc_housing_grading  191 non-null    object                   
 8   qc_housing_remark   127 non-null    object                   
 9   status              191 non-null    object                   
 10  active              191 non-null    bool                     
 11  created_at         

In [75]:
merged_data_aft21 = pd.concat([merged_data_b421, filtered_non_nan_data_api_key_2], ignore_index=True)

# Verify the merge
print(merged_data_aft21.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype                    
---  ------              --------------  -----                    
 0   session_id          246 non-null    object                   
 1   api_key_id          246 non-null    int64                    
 2   imei                246 non-null    int64                    
 3   ai_screen_grading   246 non-null    object                   
 4   qc_screen_grading   246 non-null    object                   
 5   qc_screen_remark    143 non-null    object                   
 6   ai_housing_grading  246 non-null    object                   
 7   qc_housing_grading  246 non-null    object                   
 8   qc_housing_remark   167 non-null    object                   
 9   status              246 non-null    object                   
 10  active              246 non-null    bool                     
 11  created_at         

In [77]:
merged_data_aft21.to_csv('/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/data_till_22Jan.csv', index=False)

### download images against new session ids

In [78]:
# Ensure 'created_at' is in datetime format
# merged_data['created_at'] = pd.to_datetime(merged_data['created_at'], errors='coerce', infer_datetime_format=True)

# Localize the comparison timestamp to the same timezone as 'created_at'
comparison_timestamp = pd.Timestamp('2025-01-21').tz_localize('UTC+08:00')

# Filter rows where 'created_at' is after January 14, 2025
filtered_merged_data = merged_data_aft21[merged_data_aft21['created_at'] > comparison_timestamp]

filtered_merged_data = filtered_merged_data.reset_index(drop=True)
filtered_merged_data

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,de825d71-bd5d-41d9-87b4-69d369ef6de2,2,354009841346060,A1,A1,NaN,A2,A3,"paint peelings, scratches on bezel",QC-Evaluated,True,2025-01-21 14:44:55.392000+08:00,2025-01-21 14:48:09.626 +0800,NaN
1,afd4349a-1b92-4bae-930f-1c120f8b15c1,2,357293092731850,A1,C1,heavy scratches,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:59:14.310000+08:00,2025-01-21 15:01:19.216 +0800,NaN
2,9744b947-f7aa-43a1-a7cf-1c4ef5d63781,2,352027111360258,A1,A1,NaN,A1,C1,very heavy back cover spots,QC-Evaluated,True,2025-01-21 14:29:53.649000+08:00,2025-01-21 14:32:00.402 +0800,NaN
3,fb9633e5-4a93-4457-b4f9-d783135fd537,2,351591110249300,C1,A1,no scratches,A2,C1,very heavy back cover spot,QC-Evaluated,True,2025-01-21 14:49:02+08:00,2025-01-21 14:50:21.097 +0800,NaN
4,94a7090f-9e22-4e1f-bd7f-f608df7c4dec,2,353418794348570,A1,A1,NaN,A1,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:32:15.348000+08:00,2025-01-21 14:34:31.063 +0800,NaN
5,c32e0927-e8aa-431d-9be2-5f657013c9d4,2,353904105654590,A1,A1,NaN,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:49:19.543000+08:00,2025-01-21 14:50:54.352 +0800,NaN
6,75fd3936-aeb3-4a43-a7a7-8acc0290a923,2,356716111111168,A1,C1,heavy scratches,A3,C1,"very heavy back cover spots, heavy scratches o...",QC-Evaluated,True,2025-01-21 14:34:52.270000+08:00,2025-01-21 14:37:33.967 +0800,NaN
7,1975a992-e98a-4c3f-88be-c9157ec76ff2,2,352141100694914,R2,R2,NaN,A3,C1,very heavy discoloration,QC-Evaluated,True,2025-01-21 15:03:15.835000+08:00,2025-01-21 15:04:42.322 +0800,NaN
8,f2468a34-4765-42c7-a7f9-c663f2906dfc,2,355066961222728,A1,A3,minor scratches,A1,C1,"dents, heavy discoloration, back cover spots",QC-Evaluated,True,2025-01-21 14:37:46.745000+08:00,2025-01-21 14:39:49.523 +0800,NaN
9,c26dfd36-c626-4601-b270-1a42b004d2de,2,350145451064280,A1,A1,NaN,A3,A2,"paint peelings, dings",QC-Evaluated,True,2025-01-21 14:54:58.607000+08:00,2025-01-21 14:58:58.055 +0800,NaN


In [ ]:
session_ids_aft21 = filtered_merged_data['session_id'].unique().tolist()

In [81]:
bucket_name = 'bbti-ai-grading-uat'  # Your S3 bucket name
s3_folder_prefix = 'input-image/'  # Prefix in the S3 bucket
local_root_folder = './session_ids_after_21'  # Local folder for new data

# Step 1: Get session IDs for January 21,22, 2025

# Step 2: Download and organize data for the filtered session IDs
download_and_organize_new_data(session_ids_aft21, bucket_name, s3_folder_prefix, local_root_folder)

Download and organization complete!


# Cleaning of old Data before 14Jan

## CSV PROCESSING FOR CLEANING DB DATA

In [13]:
import pandas as pd

In [68]:
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          381 non-null    object 
 1   api_key_id          381 non-null    int64  
 2   imei                381 non-null    float64
 3   ai_screen_grading   129 non-null    object 
 4   qc_screen_grading   125 non-null    object 
 5   qc_screen_remark    78 non-null     object 
 6   ai_housing_grading  129 non-null    object 
 7   qc_housing_grading  125 non-null    object 
 8   qc_housing_remark   74 non-null     object 
 9   status              346 non-null    object 
 10  active              381 non-null    bool   
 11  created_at          381 non-null    object 
 12  updated_at          381 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 39.2+ KB


In [72]:
unique_session_ids_count = df['imei'].nunique()
print(f"Number of unique session_ids: {unique_session_ids_count}")

Number of unique session_ids: 144


In [ ]:
file_path = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/ai_session_202501101733.csv"
df = pd.read_csv(file_path)

# Convert the 'created_at' column to datetime for filtering
# df['created_at'] = pd.to_datetime(df['created_at'])

# df = df[df['created_at'] > '2025-01-06'].reset_index(drop=True)

# List of columns to check for non-empty values
columns_to_check = [
    'ai_screen_grading', 
    'qc_screen_grading', 
    'qc_screen_remark', 
    'ai_housing_grading', 
    'qc_housing_grading', 
    'qc_housing_remark'
]

# Create a mask to check if any of the specified columns have data
df['has_data'] = df[columns_to_check].notna().any(axis=1)

# Sort the DataFrame by the 'has_data' column in descending order
# This ensures that rows with data come first within the same session_id
df_sorted = df.sort_values(by=['imei', 'has_data'], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (which will have data if it exists)
df_clean1 = df_sorted.drop_duplicates(subset='imei', keep='first')

# Drop the temporary 'has_data' column
df_clean1 = df_clean1.drop(columns=['has_data'])
df_clean1 = df_clean1.reset_index(drop=True)

df_clean_uniqueImei = df_clean1.dropna(subset=['qc_screen_grading', 'qc_housing_grading'], how='all')

# Reset the index after cleaning
df_clean_uniqueImei = df_clean_uniqueImei.reset_index(drop=True)

In [93]:
output_file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_df_clean_uniqueImei.csv'  # Replace with desired output file path
df_clean_uniqueImei.to_csv(output_file_path, index=False)

In [76]:
df_sorted

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at,has_data
249,15b5ce78-e09a-4454-ac98-30c07acc9ecd,2,3.500230e+14,C1,C1,NaN,C1,A2,scratches on bezel,QC-Evaluated,True,2025-01-08 09:42:15.982 +0800,2025-01-08 09:44:55.969 +0800,NaN,True
250,2e979dab-5925-4762-88e9-b62084737e5b,2,3.500230e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-08 09:42:15.929 +0800,2025-01-08 09:42:15.929 +0800,NaN,False
130,76a7cd2e-13a3-4020-9133-1a9f20cc4976,2,3.500600e+14,A2,A1,no scratches,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-07 10:25:04.491 +0800,2025-01-07 10:29:32.369 +0800,NaN,True
74,50215601-b6d1-4fca-b32e-a4cd62799b0b,2,3.501600e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-06 10:30:09.519 +0800,2025-01-06 10:30:09.519 +0800,NaN,False
77,2078fe67-ee6a-4cff-83b4-c348c709fa98,2,3.501600e+14,NaN,NaN,NaN,NaN,NaN,NaN,QC-Evaluated,True,2025-01-06 10:30:09.570 +0800,2025-01-06 10:34:10.224 +0800,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,09a7311b-45c3-4579-a8a6-a7797f19979c,2,8.648250e+14,C1,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-07 11:02:33.857 +0800,2025-01-07 11:03:50.118 +0800,NaN,True
177,76968b8d-d271-4e7d-9dc0-02d008563c7b,2,8.648250e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-07 11:00:25.365 +0800,2025-01-07 11:00:25.365 +0800,NaN,False
185,b0ffd4c7-901d-4973-9365-3a614d927725,2,8.648250e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-07 11:02:33.805 +0800,2025-01-07 11:02:33.805 +0800,NaN,False
208,30c4eee3-7399-4b01-9a1c-d10ff8758d60,2,8.699720e+14,A3,A1,no scratches,C1,A3,heavy scratches,QC-Evaluated,True,2025-01-07 11:36:22.405 +0800,2025-01-07 11:38:12.295 +0800,NaN,True


## DF WITH UNIQUE IMEI

In [77]:
df_clean1

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,15b5ce78-e09a-4454-ac98-30c07acc9ecd,2,3.500230e+14,C1,C1,NaN,C1,A2,scratches on bezel,QC-Evaluated,True,2025-01-08 09:42:15.982 +0800,2025-01-08 09:44:55.969 +0800,NaN
1,76a7cd2e-13a3-4020-9133-1a9f20cc4976,2,3.500600e+14,A2,A1,no scratches,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-07 10:25:04.491 +0800,2025-01-07 10:29:32.369 +0800,NaN
2,50215601-b6d1-4fca-b32e-a4cd62799b0b,2,3.501600e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-06 10:30:09.519 +0800,2025-01-06 10:30:09.519 +0800,NaN
3,6bb4520f-5621-4e62-b346-4a79964ecf32,2,3.503900e+14,A2,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-08 09:14:24.247 +0800,2025-01-08 09:17:24.452 +0800,NaN
4,69230ccb-f9e2-4552-b6c0-fbd93a71245a,2,3.505890e+14,A2,A2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-10 10:21:10.593 +0800,2025-01-10 10:24:21.130 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,d8b59f85-0e1e-4dc9-8198-41ed92bfc476,2,8.628560e+14,A2,A1,micro scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-09 10:57:45.452 +0800,2025-01-09 10:59:25.435 +0800,NaN
139,a35d8325-e8e1-4f9f-bbb2-8285170b8c8b,2,8.630750e+14,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-07 11:40:10.005 +0800,2025-01-07 11:41:35.798 +0800,NaN
140,95a3e6ad-95ae-4fba-976f-fbf7af68a1bc,2,8.646320e+14,NaN,NaN,NaN,NaN,NaN,NaN,Open,True,2025-01-10 16:08:30.515 +0800,2025-01-10 16:08:30.515 +0800,NaN
141,6e747a24-c028-4d37-b71e-712ada1c3898,2,8.648250e+14,A3,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-07 11:00:25.415 +0800,2025-01-07 11:02:17.214 +0800,NaN


### DF AFTER REMOVING NAN ROWS FOR QC GRADING

In [86]:
df_clean_uniqueImei

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
0,15b5ce78-e09a-4454-ac98-30c07acc9ecd,2,3.500230e+14,C1,C1,NaN,C1,A2,scratches on bezel,QC-Evaluated,True,2025-01-08 09:42:15.982 +0800,2025-01-08 09:44:55.969 +0800,NaN
1,76a7cd2e-13a3-4020-9133-1a9f20cc4976,2,3.500600e+14,A2,A1,no scratches,C1,A3,scratches on bezel,QC-Evaluated,True,2025-01-07 10:25:04.491 +0800,2025-01-07 10:29:32.369 +0800,NaN
2,6bb4520f-5621-4e62-b346-4a79964ecf32,2,3.503900e+14,A2,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-08 09:14:24.247 +0800,2025-01-08 09:17:24.452 +0800,NaN
3,69230ccb-f9e2-4552-b6c0-fbd93a71245a,2,3.505890e+14,A2,A2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-10 10:21:10.593 +0800,2025-01-10 10:24:21.130 +0800,NaN
4,dbe72c84-a7a4-4489-8aeb-3425345a6a1f,2,3.506670e+14,C1,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-10 10:34:42.633 +0800,2025-01-10 10:36:15.648 +0800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,89987845-d92f-4506-8644-243a3e1c726a,2,8.615850e+14,A1,A1,NaN,C1,A1,no dent,QC-Evaluated,True,2025-01-07 10:30:26.165 +0800,2025-01-07 10:32:01.551 +0800,NaN
101,d8b59f85-0e1e-4dc9-8198-41ed92bfc476,2,8.628560e+14,A2,A1,micro scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-09 10:57:45.452 +0800,2025-01-09 10:59:25.435 +0800,NaN
102,a35d8325-e8e1-4f9f-bbb2-8285170b8c8b,2,8.630750e+14,C1,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-07 11:40:10.005 +0800,2025-01-07 11:41:35.798 +0800,NaN
103,6e747a24-c028-4d37-b71e-712ada1c3898,2,8.648250e+14,A3,A1,no scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-07 11:00:25.415 +0800,2025-01-07 11:02:17.214 +0800,NaN


In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/ai_session_202501101733.csv"
df = pd.read_csv(file_path)

# List of columns to check for non-empty values
columns_to_check = [
    'ai_screen_grading', 
    'qc_screen_grading', 
    'qc_screen_remark', 
    'ai_housing_grading', 
    'qc_housing_grading', 
    'qc_housing_remark'
]

# Create a mask to check if any of the specified columns have data
df['has_data'] = df[columns_to_check].notna().any(axis=1)

# Sort the DataFrame by the 'has_data' column in descending order
# This ensures that rows with data come first within the same session_id
df_clean1 = df.sort_values(by=['session_id', 'has_data'], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (which will have data if it exists)
df_clean2 = df_clean1.drop_duplicates(subset='session_id', keep='first')

# Drop the temporary 'has_data' column
df_clean2 = df_clean2.drop(columns=['has_data'])

# Save the cleaned DataFrame back to a new CSV file
output_file_path = 'cleaned_file.csv'  # Replace with desired output file path
df_clean2.to_csv(output_file_path, index=False)

print("Processing complete. Cleaned file saved as:", output_file_path)

In [ ]:


output_file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file.csv'  # Replace with desired output file path
df_clean2.to_csv(output_file_path, index=False)

In [80]:
df_clean2_ = pd.read_csv(output_file_path)
# Remove rows where both 'qc_screen_grading' and 'qc_housing_grading' are null
df_clean3 = df_clean2_.dropna(subset=['qc_screen_grading', 'qc_housing_grading'], how='all')

# # Reset the index after cleaning
# df_clean3 = df_clean3.reset_index(drop=True)
# df_clean3

In [ ]:
output_file_path = '/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/data/cleaned_file_prev_method_nonNan.csv'  # Replace with desired output file path
df_clean3.to_csv(output_file_path, index=False)

In [99]:
df_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, 0 to 129
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          125 non-null    object 
 1   api_key_id          125 non-null    int64  
 2   imei                125 non-null    int64  
 3   ai_screen_grading   112 non-null    object 
 4   qc_screen_grading   125 non-null    object 
 5   qc_screen_remark    78 non-null     object 
 6   ai_housing_grading  112 non-null    object 
 7   qc_housing_grading  125 non-null    object 
 8   qc_housing_remark   74 non-null     object 
 9   status              125 non-null    object 
 10  active              125 non-null    bool   
 11  created_at          125 non-null    object 
 12  updated_at          125 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 13.8+ KB


In [98]:
df['imei']

0      3.563690e+14
1      3.572350e+14
2      3.572300e+14
3      3.572350e+14
4      3.572350e+14
           ...     
375    3.546200e+14
376    3.558220e+14
377    3.539090e+14
378    3.546200e+14
379    3.586120e+14
Name: imei, Length: 380, dtype: float64

In [90]:
df_clean_uniqueImei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          105 non-null    object 
 1   api_key_id          105 non-null    int64  
 2   imei                105 non-null    float64
 3   ai_screen_grading   94 non-null     object 
 4   qc_screen_grading   105 non-null    object 
 5   qc_screen_remark    64 non-null     object 
 6   ai_housing_grading  94 non-null     object 
 7   qc_housing_grading  105 non-null    object 
 8   qc_housing_remark   61 non-null     object 
 9   status              105 non-null    object 
 10  active              105 non-null    bool   
 11  created_at          105 non-null    object 
 12  updated_at          105 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 10.9+ KB


In [91]:
df_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, 0 to 129
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          125 non-null    object 
 1   api_key_id          125 non-null    int64  
 2   imei                125 non-null    int64  
 3   ai_screen_grading   112 non-null    object 
 4   qc_screen_grading   125 non-null    object 
 5   qc_screen_remark    78 non-null     object 
 6   ai_housing_grading  112 non-null    object 
 7   qc_housing_grading  125 non-null    object 
 8   qc_housing_remark   74 non-null     object 
 9   status              125 non-null    object 
 10  active              125 non-null    bool   
 11  created_at          125 non-null    object 
 12  updated_at          125 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 13.8+ KB


In [87]:
# Merge the two dataframes with an indicator column
diff_df = pd.merge(df_clean_uniqueImei, df_clean3, how='outer', indicator=True)

# Filter rows that are not present in both dataframes
unique_rows = diff_df[diff_df['_merge'] != 'both']

print("Rows that are unique to either dataframe:")
print(unique_rows)

# Optional: Split unique rows into separate DataFrames for each dataframe
unique_to_df1 = unique_rows[unique_rows['_merge'] == 'left_only']
unique_to_df2 = unique_rows[unique_rows['_merge'] == 'right_only']

Rows that are unique to either dataframe:
                               session_id  api_key_id          imei  \
0    04688db2-3a16-40d6-929b-671d70f1202b           2  3.508370e+14   
1    04688db2-3a16-40d6-929b-671d70f1202b           2  3.508374e+14   
2    04fad5ad-ac47-4944-910d-07276eec37c5           2  3.540030e+14   
3    04fad5ad-ac47-4944-910d-07276eec37c5           2  3.540031e+14   
4    06085b81-ab2e-4ec1-b46f-c5b71b28b2e1           2  3.546262e+14   
..                                    ...         ...           ...   
225  fd246594-8291-474f-9bdb-f30e6b181871           2  3.569150e+14   
226  fd246594-8291-474f-9bdb-f30e6b181871           2  3.569151e+14   
227  fe12b025-bb4d-465e-868b-60d77f118a9f           2  3.546262e+14   
228  fe8c13ad-e313-4735-80b8-176c3bd697c8           2  3.550180e+14   
229  fe8c13ad-e313-4735-80b8-176c3bd697c8           2  3.550182e+14   

    ai_screen_grading qc_screen_grading qc_screen_remark ai_housing_grading  \
0                  C1     

In [89]:
unique_to_df2

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at,_merge
1,04688db2-3a16-40d6-929b-671d70f1202b,2,3.508374e+14,C1,C1,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-08 09:09:32.420000+08:00,2025-01-08 09:11:19.850 +0800,NaN,right_only
3,04fad5ad-ac47-4944-910d-07276eec37c5,2,3.540031e+14,A3,A3,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-10 10:47:47.786000+08:00,2025-01-10 10:49:32.270 +0800,NaN,right_only
4,06085b81-ab2e-4ec1-b46f-c5b71b28b2e1,2,3.546262e+14,A2,A1,no scratches,C1,A1,no scratches,QC-Evaluated,True,2025-01-10 10:31:01.279000+08:00,2025-01-10 10:32:27.423 +0800,NaN,right_only
5,07631b81-c317-4179-943f-b550b8e2f934,2,3.590568e+14,C1,A1,no scratches,C1,A3,heavy scratches on bezel,QC-Evaluated,True,2025-01-10 10:38:37.563000+08:00,2025-01-10 10:40:48.739 +0800,NaN,right_only
8,08e30cbb-1f87-48bc-8622-3a68b1851ca3,2,3.565651e+14,A3,A3,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-07 10:56:24.909000+08:00,2025-01-07 10:58:30.916 +0800,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,fb03b93d-cd42-4a42-9ea8-3182cffb164f,2,3.548627e+14,A2,C1,very heavy scratches,C1,C1,NaN,QC-Evaluated,True,2025-01-10 10:27:38.943000+08:00,2025-01-10 10:28:48.772 +0800,NaN,right_only
224,fccf3c74-357e-4e35-89f8-40f0c9d0cfac,2,3.533247e+14,R2,A3,heavy scratches,C1,C1,very heavy back cover spot,QC-Evaluated,True,2025-01-07 10:34:02.917000+08:00,2025-01-07 10:36:23.694 +0800,NaN,right_only
226,fd246594-8291-474f-9bdb-f30e6b181871,2,3.569151e+14,R2,R2,NaN,C1,C1,NaN,QC-Evaluated,True,2025-01-08 10:00:42.984000+08:00,2025-01-08 10:01:42.210 +0800,NaN,right_only
227,fe12b025-bb4d-465e-868b-60d77f118a9f,2,3.546262e+14,A1,A1,NaN,C1,A2,minor ding,QC-Evaluated,True,2025-01-10 10:32:56.167000+08:00,2025-01-10 10:34:16.952 +0800,NaN,right_only


In [84]:
df_clean2_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          136 non-null    object 
 1   api_key_id          136 non-null    int64  
 2   imei                136 non-null    int64  
 3   ai_screen_grading   123 non-null    object 
 4   qc_screen_grading   125 non-null    object 
 5   qc_screen_remark    78 non-null     object 
 6   ai_housing_grading  123 non-null    object 
 7   qc_housing_grading  125 non-null    object 
 8   qc_housing_remark   74 non-null     object 
 9   status              136 non-null    object 
 10  active              136 non-null    bool   
 11  created_at          136 non-null    object 
 12  updated_at          136 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 14.1+ KB


In [82]:
df_clean3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, 0 to 129
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          125 non-null    object 
 1   api_key_id          125 non-null    int64  
 2   imei                125 non-null    int64  
 3   ai_screen_grading   112 non-null    object 
 4   qc_screen_grading   125 non-null    object 
 5   qc_screen_remark    78 non-null     object 
 6   ai_housing_grading  112 non-null    object 
 7   qc_housing_grading  125 non-null    object 
 8   qc_housing_remark   74 non-null     object 
 9   status              125 non-null    object 
 10  active              125 non-null    bool   
 11  created_at          125 non-null    object 
 12  updated_at          125 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 13.8+ KB


In [40]:
df_clean3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   session_id          127 non-null    object 
 1   api_key_id          127 non-null    int64  
 2   imei                127 non-null    int64  
 3   ai_screen_grading   113 non-null    object 
 4   qc_screen_grading   127 non-null    object 
 5   qc_screen_remark    80 non-null     object 
 6   ai_housing_grading  113 non-null    object 
 7   qc_housing_grading  127 non-null    object 
 8   qc_housing_remark   76 non-null     object 
 9   status              127 non-null    object 
 10  active              127 non-null    bool   
 11  created_at          127 non-null    object 
 12  updated_at          127 non-null    object 
 13  deleted_at          0 non-null      float64
dtypes: bool(1), float64(1), int64(2), object(10)
memory usage: 13.1+ KB


In [83]:
df_clean3[df_clean3['imei'].duplicated(keep=False)]

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at


In [43]:
unique_session_ids_count = df['session_id'].nunique()
print(f"Number of unique session_ids: {unique_session_ids_count}")

unique_session_ids_count = df['imei'].nunique()
print(f"Number of unique imei: {unique_session_ids_count}")

Number of unique session_ids: 412
Number of unique imei: 176


In [34]:
df_clean2_ = pd.read_csv(output_file_path)

# Find duplicate rows based on the 'imei' column
duplicate_rows = df_clean2_[df_clean2_['imei'].duplicated(keep=False)]
duplicate_rows

,session_id,api_key_id,imei,ai_screen_grading,qc_screen_grading,qc_screen_remark,ai_housing_grading,qc_housing_grading,qc_housing_remark,status,active,created_at,updated_at,deleted_at
40,b2918e06-022f-4df7-bf1a-f8ae767552e5,1,355705441601221,A3,NaN,NaN,C1,NaN,NaN,Grading_Sucess,True,2025-01-07 11:12:21.489000+08:00,2025-01-07 11:12:58.929 +0800,NaN
45,3b607741-a7fa-4286-8c8b-35b579659d8a,1,355705441601221,A3,C1,thhh,C1,C1,NaN,QC-Evaluated,True,2025-01-07 11:37:59.258000+08:00,2025-01-07 11:39:03.568 +0800,NaN
110,da152709-96dc-47dd-b522-ae69fb136287,1,354619649717690,R2,NaN,NaN,R2,NaN,NaN,Grading_Sucess,True,2025-01-10 15:42:40.747000+08:00,2025-01-10 15:44:26.803 +0800,NaN
128,a738faff-ca2c-4e15-87ed-a77f2e2d1805,1,354619649717690,A1,NaN,NaN,R2,NaN,NaN,Grading_Sucess,True,2025-01-10 15:47:32.462000+08:00,2025-01-10 15:48:32.481 +0800,NaN
130,96295f03-e206-4979-821f-f48c1442eb40,1,354619649717690,A3,NaN,NaN,C1,NaN,NaN,Grading_Sucess,True,2025-01-10 14:15:12.240000+08:00,2025-01-10 14:21:14.720 +0800,NaN
132,258de34d-550f-4590-aff7-6185f00fee81,1,354619649717690,A2,NaN,NaN,C1,NaN,NaN,Grading_Sucess,True,2025-01-10 14:21:45.652000+08:00,2025-01-10 14:22:52.727 +0800,NaN
133,17569924-b893-4f3d-903b-417d2c4269dc,1,354619649717690,A3,NaN,NaN,A1,NaN,NaN,Grading_Sucess,True,2025-01-10 14:47:16.018000+08:00,2025-01-10 14:52:23.611 +0800,NaN
134,9f8a91c2-0401-48c5-8baa-ce6759279a90,1,354619649717690,R2,NaN,NaN,C1,NaN,NaN,Grading_Sucess,True,2025-01-10 16:05:33.898000+08:00,2025-01-10 16:07:44.430 +0800,NaN


In [35]:
import boto3
import os

# Initialize the S3 client
s3 = boto3.client('s3')

# Define the bucket name and folder
bucket_name = 'bbti-ai-grading-uat'  # Replace with your bucket name
folder_name = 'input-image/'  # Folder to download
local_download_path = './downloaded_input_image'  # Local folder to save files

# Ensure local download directory exists
os.makedirs(local_download_path, exist_ok=True)

# List objects in the folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

if 'Contents' in response:
    print(f"Downloading files from folder: {folder_name}")
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Skip the folder itself (empty key after prefix)
        if file_key == folder_name:
            continue
        
        # Define local file path
        local_file_path = os.path.join(local_download_path, os.path.relpath(file_key, folder_name))
        
        # Ensure local subdirectories exist
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        
        # Download the file
        print(f"Downloading {file_key} to {local_file_path}")
        s3.download_file(bucket_name, file_key, local_file_path)
    print("Download complete!")
else:
    print(f"No files found in folder: {folder_name}")

Download complete!


# Organize data

In [84]:
import os
import shutil

# Define the paths
main_folder = "/Users/sabainaharoon/Documents/compasia/ai_grading/grading_system/testing_grading_accuracy/organized_cleaned_images"  # Replace with the path to your main folder
output_folder = "/Users/sabainaharoon/Downloads/production_data"  # Replace with the desired output folder path

# Define the new folder structure
categories = {
    "FRONT": ["FRONT"],
    "BACK": ["BACK"],
    "TOP_BOTTOM": ["TOP", "BOTTOM"],
    "LEFT_RIGHT": ["LEFT", "RIGHT"],
}

# Create the new folders
for category in categories.keys():
    os.makedirs(os.path.join(output_folder, category), exist_ok=True)

# check if output folder exists, else mkdir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Traverse the main folder
for session_id in os.listdir(main_folder):
    session_path = os.path.join(main_folder, session_id)
    if os.path.isdir(session_path):  # Check if it's a folder
        for image_file in os.listdir(session_path):
            for category, keywords in categories.items():
                if any(keyword in image_file for keyword in keywords):
                    # Move the image to the corresponding folder
                    source_path = os.path.join(session_path, image_file)
                    destination_path = os.path.join(output_folder, category, image_file)
                    shutil.copy2(source_path, destination_path)  # Use shutil.move() to move instead of copy
                    break

print("Images have been reorganized into the new folder structure.")

Images have been reorganized into the new folder structure.


In [ ]:
df = df_246devices_info.reset_index()
df.columns = ['Screen Grade', 'Count']
df

,Screen Grade,Count
0,A1,161
1,C1,35
2,R2,23
3,A3,16
4,A2,11
